In [2]:
import numpy as np
import sympy as sym
import numba
import pydae.build as db

In [32]:
g01.keys()

dict_keys(['name', 'bus', 'S_b', 'X_d', 'X1d', 'T1d0', 'X_q', 'X1q', 'T1q0', 'R_a', 'X_l', 'H', 'D', 'Omega_b', 'omega_s', 'v_0', 'theta_0', 'K_a', 'T_r', 'v_pss', 'P_t', 'Q_t'])

In [56]:
g01 = {'name':'g01','bus':1,'S_b':2200e6,'X_d':1.81,'X1d':0.3,'T1d0':8.0,   # synnchronous machine d-axis parameters
          'X_q':1.76,'X1q':0.65,'T1q0':1.0,  # synnchronous machine q-axis parameters
          'R_a':0.003,'X_l': 0.05, 
          'H':3.5,'D':1.0,
          'Omega_b':2*np.pi*50,'omega_s':1.0,
          'v_0':1.0,'theta_0':0.0,
          'K_a':100, 'T_r':0.1, 'v_pss':0.0,
           'p_t':0.8, 'q_t':0.2 }


data = {
"system":{"S_b":2200e6,"f_hz":50,"model_type":"ode"},
"buses": [{"bus":"1"},{"bus":"0"}],
"lines":     [  
               {"bus_j":"1", "bus_k":"0", "R":0.01, "X":0.05, "B":0.0},
             ],
"loads" : [
        ],
"syns" : [
          g01,
                 ],
"grid_formers" : [
                  #{"bus": "12","V_phph":20.0e3, "deg":0.0},
                 ]}

In [57]:
data

{'system': {'S_b': 2200000000.0, 'f_hz': 50, 'model_type': 'ode'},
 'buses': [{'bus': '1'}, {'bus': '0'}],
 'lines': [{'bus_j': '1', 'bus_k': '0', 'R': 0.01, 'X': 0.05, 'B': 0.0}],
 'loads': [],
 'syns': [{'name': 'g01',
   'bus': 1,
   'S_b': 2200000000.0,
   'X_d': 1.81,
   'X1d': 0.3,
   'T1d0': 8.0,
   'X_q': 1.76,
   'X1q': 0.65,
   'T1q0': 1.0,
   'R_a': 0.003,
   'X_l': 0.05,
   'H': 3.5,
   'D': 1.0,
   'Omega_b': 314.1592653589793,
   'omega_s': 1.0,
   'v_0': 1.0,
   'theta_0': 0.0,
   'K_a': 100,
   'T_r': 0.1,
   'v_pss': 0.0,
   'p_t': 0.8,
   'q_t': 0.2}],
 'grid_formers': []}

In [59]:
sin = sym.sin
cos = sym.cos

# AVR
#dv_c =   (v_1 - v_c)/T_r
#g_7 = K_a*(v_ref - v_c + v_pss) - v_f 


f_syn = []
x_syn = []
g_syn = []
y_syn_ini = []
y_syn_run = []

syn = data['syns'][0]
name = syn['name']
x_list = ['delta','omega','e1q','e1d']  
y_ini_list = ['i_d','i_q','v_1','theta_1','p_m','v_f'] 
y_run_list = ['i_d','i_q','v_1','theta_1','P_t','Q_t']    

# synchronous generator 4rd order as in Milano's book
delta,omega,e1q,e1d = sym.symbols([item + f"_{syn['name']}" for item in x_list],real=True)
i_d,i_q,v_1,theta_1,p_m,v_ref = sym.symbols([item + f"_{syn['name']}" for item in y_ini_list],real=True)
## parameters:
params_vsc_filter = {}
for item in ['S_b', 'X_d', 'X1d', 'T1d0', 'X_q', 'X1q', 'T1q0', 'R_a', 'X_l', 'H', 'D', 'Omega_b', 'omega_s']:
    params_vsc_filter.update({f'{item}_{name}':syn[item]})
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())
## inputs:
u_vsc_filter = {}
for item in ['p_t', 'q_t']:#,'eta_D','eta_Q']:
    u_vsc_filter.update({f'{item}_{name}':syn[item]})
    exec(f"{item} = sym.Symbol('{item}_{name}', real=True)",globals())

v_f,p_m,omega_coi = sym.symbols([item + f"_{syn['name']}" for item in ['v_f','p_m','omega_coi']],real=True)
v_d = v_1*sin(delta - theta_1) 
v_q = v_1*cos(delta - theta_1) 

p_e = i_d*(v_d + R_a*i_d) + i_q*(v_q + R_a*i_q) 

ddelta = Omega_b*(omega - omega_coi)
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_coi))
de1q = 1/T1d0*(-e1q - (X_d - X1d)*i_d + v_f)
de1d = 1/T1q0*(-e1d + (X_q - X1q)*i_q)

g_i_d = v_q + R_a*i_q + X1d*i_d - e1q
g_i_q = v_d + R_a*i_d - X1q*i_q - e1d
g_p_t = i_d*v_d + i_q*v_q - p_t
g_q_t = i_d*v_q - i_q*v_d - q_t

f_syn += [ddelta,domega,de1q,de1d]
x_syn += [ delta, omega, e1q, e1d]
g_syn += [g_i_d,g_i_q,g_p_t,g_q_t]
y_syn_ini += [ i_d, i_q, p_m, v_f]
y_syn_run += [ i_d, i_q, p_t, q_t]

In [53]:
domega

(-D_g01*(-omega_coi_g01 + omega_g01) - i_d_g01*(R_a_g01*i_d_g01 + v_1_g01*sin(delta_g01 - theta_1_g01)) - i_q_g01*(R_a_g01*i_q_g01 + v_1_g01*cos(delta_g01 - theta_1_g01)) + p_m_g01)/(2*H_g01)

In [52]:
[item + f"_{syn['name']}" for item in x_list]

['delta_g01', 'omega_g01', 'e1q_g01', 'e1d_g01']

In [30]:
g_3 = P_t - (v_1*v_0*sin(theta_1 - theta_0))/X_l
g_4 = Q_t + (v_1*v_0*cos(theta_1 - theta_0))/X_l - v_1**2/X_l

delta_g01

\begin{eqnarray}
\dot \delta &=& \Omega_b \left(\omega - \omega_s\right)\\
\dot \omega &=& 1/(2 H) \left(p_m - p_e - D (\omega - \omega_s) \right)\\
\end{eqnarray}

$$ \sf
\Omega_{b} \left(\sf \omega - \omega_{s}\right)
$$

## System definition 

In [2]:
params = {'X_d':1.81,'X1d':0.3,'T1d0':8.0,   # synnchronous machine d-axis parameters
          'X_q':1.76,'X1q':0.65,'T1q0':1.0,  # synnchronous machine q-axis parameters
          'R_a':0.003,'X_l': 0.05, 
          'H':3.5,'D':1.0,
          'Omega_b':2*np.pi*50,'omega_s':1.0,
          'v_0':1.0,'theta_0':0.0,
          'K_a':100, 'T_r':0.1, 'v_pss':0.0}


u_ini_dict = {'P_t':0.8, 'Q_t':0.2}  # for the initialization problem
u_run_dict = {'p_m':0.8,'v_ref':1.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['delta','omega','e1q','e1d','v_c']    # [inductor current, PI integrator]
y_ini_list = ['i_d','i_q','v_1','theta_1','p_m','v_ref','v_f'] # for the initialization problem
y_run_list = ['i_d','i_q','v_1','theta_1','P_t','Q_t',  'v_f'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}

exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [3]:
v_d = v_1*sin(delta - theta_1) 
v_q = v_1*cos(delta - theta_1) 

p_e = i_d*(v_d + R_a*i_d) + i_q*(v_q + R_a*i_q) 


ddelta = Omega_b*(omega - omega_s)
domega = 1/(2*H)*(p_m - p_e - D*(omega - omega_s))
de1q = 1/T1d0*(-e1q - (X_d - X1d)*i_d + v_f)
de1d = 1/T1q0*(-e1d + (X_q - X1q)*i_q)
dv_c =   (v_1 - v_c)/T_r

g_1 = v_q + R_a*i_q + X1d*i_d - e1q
g_2 = v_d + R_a*i_d - X1q*i_q - e1d
g_3 = P_t - (v_1*v_0*sin(theta_1 - theta_0))/X_l
g_4 = Q_t + (v_1*v_0*cos(theta_1 - theta_0))/X_l - v_1**2/X_l
g_5 = i_d*v_d + i_q*v_q - P_t
g_6 = i_d*v_q - i_q*v_d - Q_t
g_7 = K_a*(v_ref - v_c + v_pss) - v_f 

h_1 = p_m

sys = {'name':'smib_milano_ex8p1_4ord_avr',
       'params':params,
       'f':[ddelta,domega,de1q,de1d,dv_c],
       'g':[g_1,g_2,g_3,g_4,g_5,g_6,g_7],
       'g_ini':[g_1,g_2],
       'x':x_list,
       'y_ini':y_ini_list,
       'y':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h':[h_1]}

sys = db.system(sys)
db.sys2num(sys)
                